In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [26]:
from cast.filters import PostFilter
from cast.filters import parse_date_facets

<IPython.core.display.Javascript object>

In [2]:
def get_facet_counts(self, filterset_class, kwargs):
    kwargs = {k: v for k, v in kwargs.items()}  # copy kwargs to avoid overwriting

    # get selected facet if set and build the facet counting queryset
    facet_counts = {}
    selected_facet = self.get_selected_facet()
    if selected_facet is not None:
        facet_counts = {"year_month": {selected_facet: 1}}
    kwargs["facet_counts"] = facet_counts
    post_filter = filterset_class(**kwargs)
    facet_queryset = (
        post_filter.qs.order_by()
        .annotate(month=TruncMonth("visible_date"))
        .values("month")
        .annotate(n=models.Count("pk"))
    )

    # build up the date facet counts for final filter pass
    year_month_counts = {}
    for row in facet_queryset:
        year_month_counts[row["month"]] = row["n"]
    return {"year_month": year_month_counts}

<IPython.core.display.Javascript object>

In [16]:
def get_selected_facet(self):
    """Return the currently selected facet. Otherwise we would see
    all date facets that are choosable if no date facet was selected
    because in the final pass over the queryset facet_counts would be
    empty and the selected facet would not be accepted because it's
    not in the fields choices."""
    data = self.request.GET or None
    if data is None:
        return None
    date_facet = data.get("date_facets")
    if date_facet is None or len(date_facet) == 0:
        return None
    return parse_date_facets(date_facet)

<IPython.core.display.Javascript object>

In [17]:
blog = Blog.objects.first()

<IPython.core.display.Javascript object>

In [18]:
class StubRequest:
    GET = {"date_facets": "2021-07"}


blog.request = StubRequest()

<IPython.core.display.Javascript object>

In [19]:
print(get_selected_facet(blog))

2021-07-01 00:00:00


<IPython.core.display.Javascript object>

In [22]:
qs = Post.objects.live().descendant_of(blog).order_by("-visible_date")

<IPython.core.display.Javascript object>

In [23]:
qs.count()

5

<IPython.core.display.Javascript object>

In [27]:
filterset = PostFilter(blog=blog, queryset=qs)

<IPython.core.display.Javascript object>

In [29]:
filterset.qs.count()

5

<IPython.core.display.Javascript object>

In [30]:
filterset.is_valid()

False

<IPython.core.display.Javascript object>

In [35]:
filterset.is_bound

False

<IPython.core.display.Javascript object>

In [38]:
qs.search("eindeutiges")

<SearchResults [<Post: post with audio>]>

<IPython.core.display.Javascript object>